# Pinhole Camera Model Practice Questions

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. 
The camera has a focal length `f` of 50mm. 
Calculate the position of the projection of this point on the image plane in meters.

In [35]:
# Your code here
import numpy as np

f = 0.05  # Example focal length in mm
D_point = np.array([10,5,20]).reshape(3,1)
def calculate_position_of_projection ( focal_lengthA, D_point):
    """
    Calculate the position of projection based on focal length and distance point.
    
    Parameters:
    Focal_length (float): The focal length of the camera.
    D_point (np.array): The distance point in 3D space.
    
    Returns:
    np.array: The position of projection in 3D space.
    """
    if focal_lengthA <= 0:
        raise ValueError("Focal length must be a positive number.")
    
    if D_point.shape != (3,1):
        raise ValueError("D_point must be a 3D point represented as a numpy array of length 3.")
    # Calculate the position of projection
    X= D_point[0, 0]
    Y= D_point[1, 0]
    Z= D_point[2, 0]
    
    if Z == 0:
        raise ValueError("Z coordinate cannot be zero (point is on the camera plane).")
    
    # Standard perspective projection formulas
    u = focal_lengthA * (X / Z)
    v = focal_lengthA * (Y / Z)
    
    return np.array([u,v,1]).reshape(3,1)

position_of_projection = calculate_position_of_projection(f, D_point)
print("Position of Projection:", position_of_projection)


Position of Projection: [[0.025 ]
 [0.0125]
 [1.    ]]


---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. 
Assume the following intrinsic parameters: 
- `s_x` = 800 pixels/meter
- `s_y` = 800 pixels/meter
- `o_x` = 320 pixels
- `o_y` = 240 pixels
What are the pixel coordinates `(u^I, v^I)`?

In [36]:
# Your code here
s_x = 800
s_y = 800
o_x = 320
o_y = 240
intrinsic_params = np.array([[s_x,0,o_x],[0,s_y,o_y],[0,0,1]]).reshape(3,3)
def POP_M_to_Pixel (position_of_projectionA, intrinsic_ParamsA):
    """
    Convert the position of projection in meters to pixel coordinates.
    
    Parameters:
    position_of_projection (np.array): The position of projection in meters.
    image_size (tuple): The size of the image in pixels (width, height).
    
    Returns:
    tuple: The pixel coordinates (u, v).
    """
    return intrinsic_ParamsA@ position_of_projectionA

pixel_coordinates = POP_M_to_Pixel(position_of_projection, intrinsic_params)

print("Pixel Coordinates:", pixel_coordinates)

Pixel Coordinates: [[340.]
 [250.]
 [  1.]]


---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

In [37]:
# Your code here
s_theta = 0
K = np.array([[s_x*f,s_theta*f,o_x],[0,s_y*f,o_y],[0,0,1]]).reshape(3,3)

print("Intrinsic Calibration Matrix:\n", K)

Intrinsic Calibration Matrix:
 [[ 40.   0. 320.]
 [  0.  40. 240.]
 [  0.   0.   1.]]


---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t_w^c = [0, 0, 10]`. [cite_start]Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point. [cite: 14, 16, 17]

In [ ]:
# Your code here
p_w = np.array([2,3,50,1]).reshape(4,1)
t_w_c = np.array([0,0,10])
r_w_c = np.array([[1,0,0],[0,1,0],[0,0,1]]).reshape(3,3)
def calc_proj_point(p_w, t_w_c, r_w_c, k):
    """
    Calculate the projection point in pixel coordinates.
    
    Parameters:
    p_w (np.array): The world point in 3D space.
    t_w_c (np.array): The translation vector from world to camera coordinates.
    r_w_c (np.array): The rotation matrix from world to camera coordinates.
    intrinsic_calibration_matrix (np.array): The intrinsic calibration matrix.
    
    Returns:
    np.array: The projection point in pixel coordinates.
    """
    extrinsic_matrix = np.hstack((r_w_c, t_w_c.reshape(3, 1)))  # Combine rotation and translation
    project_P = k @ extrinsic_matrix @ p_w
    
    return project_P

p_w_3 = np.array([2,3,50]).reshape(3,1)  # 3D point in world coordinates
a= calculate_position_of_projection(50, p_w_3)  # Calculate position of projection

print ("cam pow:", a)
projection_point4 = calc_proj_point(p_w, t_w_c, r_w_c, K)/60 # Normalize by the third element
print ("Projection Point in Pixel Coordinates:", projection_point4)


cam pow: [[2.]
 [3.]
 [1.]]
Projection Point in Pixel Coordinates: [[321.33333333]
 [242.        ]
 [  1.        ]]


---

## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

In [53]:
# Your code here
point_A = np.array([400,350])
principal_point = np.array([320,240])
a1 = 0.0001
a2 = 0.0000002
def corrected_pixel_coordinates(Point_aA, principal_pointA, a1A, a2A):
    """
    Correct pixel coordinates to ensure they are within the image size.
    
    Parameters:
    pixel_coordinates (np.array): The pixel coordinates to correct.
    image_size (tuple): The size of the image in pixels (width, height).
    
    Returns:
    np.array: The corrected pixel coordinates.
    """
    r_2 = (Point_aA[0] - principal_pointA[0])**2 + (Point_aA[1] - principal_pointA[1])**2
    r_4 = r_2**2
    u_corrected = (1+a1A*r_2+a2A*r_4)*(Point_aA[0] - principal_pointA[0]) + principal_pointA[0]
    v_corrected = (1+a1A*r_2+a2A*r_4)*(Point_aA[1] - principal_pointA[1]) + principal_pointA[1]
    return np.array([u_corrected, v_corrected]).reshape(2, 1)

corrected_coordinates = corrected_pixel_coordinates(point_A, principal_point, a1, a2)
print("Corrected Pixel Coordinates:", corrected_coordinates)

Corrected Pixel Coordinates: [[6024.]
 [8083.]]
